Verify identical embeddings from local Chroma DB vs. Elastic Search

In [1]:
from CreateDocuments import load_documents
import RAG_utils

import pandas as pd
import tqdm
%load_ext autoreload
%autoreload 2

In [6]:
# db = RAG_utils.create_chroma_db(doc_path='documents/train_size_1000_overlap_200_documents.pkl', collection_name='train_full')
db = RAG_utils.access_chroma_db(collection_name='train_full')
db._collection.count()

Accessed collection: train_full


45877

In [7]:
import json
f = open('context.json',) 
aws_embeddings = json.load(f)

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [9]:
def compare_lists_rounded(list1, list2, precision):
    rounded_list1 = [round(x, precision) if isinstance(x, float) else x for x in list1]
    rounded_list2 = [round(x, precision) if isinstance(x, float) else x for x in list2]
    return rounded_list1 == rounded_list2

In [10]:
precision = 4
chroma_docs = []
for i, doc in enumerate(aws_embeddings):
    
    doc_id = doc['_source']['doc_id']
    vector = doc['_source']['vector']
    aws_string = doc['_source']['sentence']
    query = aws_string
    vector = model.embed_query(query)
    x = db._collection.query(vector, n_results=1, include=['documents', 'embeddings'])
    chroma_string = x['documents'][0][0]
    chroma_embedding = x['embeddings'][0][0]
    #print(len(chroma_embedding))
    chroma_docs.append(x)

    # make sure retrieved doc is same doc from es
    assert aws_string == chroma_string, f"===AWS===\n {aws_string}\n\n ===Chroma=== \n{chroma_string}"
    # check if the embeddings are the same
    assert compare_lists_rounded(chroma_embedding, vector, precision), f"{i}"
 
    

In [11]:
x['documents'][0]

['- 24337 W Boulevard De John3 beds, 2 baths\n1,734 sqft, 22,651 sqft lot, built in 1974\n- 3304 Mistflower Ln4 beds, 5 baths\n3,114 sqft, 10,123 sqft lot, built in 2003\nRentals\n- 24330 W Boulevard De John3 beds, 2 baths\n-- sqft, -- sqft lot, built in --\n- 3412 Minito Ct5 beds, 5 baths\n4,950 sqft, -- sqft lot, built in 2005\n- 3007 Saganashkee Ln2 beds, 3 baths\n1,850 sqft, -- sqft lot, built in 2003\n- 3007 Saganashkee Ln # 30072 beds, 2.5 baths\n1,850 sqft, -- sqft lot, built in 2003\n- 3443 Breitwieser Ln4 beds, 3.5 baths\n3,118 sqft, 10,100 sqft lot, built in 2001\n- 2809 Wheatland Ct2 beds, 2.5 baths\n1,550 sqft, -- sqft lot, built in 1998\n- 2817 Ledgestone Ct3 beds, 2.5 baths\n1,500 sqft, -- sqft lot, built in 1998\n- 2812 Cantore Rd3 beds, 2.5 baths\n1,502 sqft, -- sqft lot, built in 1997\n- 2876 Cedar Glade Dr3 beds, 2.5 baths\n1,624 sqft, -- sqft lot, built in 1997\n- 11336 Highland Dr S4 beds, 2.5 baths\n2,200 sqft, -- sqft lot, built in 2005\n- 11338 Highland Dr S4 bed

In [12]:
db._collection.get(include=['documents','embeddings'], limit=1)

{'ids': ['00062acb-0d56-4d2b-b928-9c7604ab3f38'],
 'embeddings': [[-0.013739907182753086,
   0.010007297620177269,
   0.01934611052274704,
   -0.037626031786203384,
   0.06661075353622437,
   0.04583337903022766,
   0.08674821257591248,
   -0.01815810427069664,
   0.002521977759897709,
   0.03281666338443756,
   -0.026633162051439285,
   0.028172729536890984,
   0.05418466404080391,
   -0.009512516669929028,
   0.0016620670212432742,
   0.01445675827562809,
   0.02871713787317276,
   -0.01855434849858284,
   -0.030500799417495728,
   0.14130988717079163,
   -0.03377174958586693,
   0.03985656425356865,
   0.02172563225030899,
   0.0010366756469011307,
   0.013550879433751106,
   0.005716213025152683,
   0.00891695823520422,
   -0.047977156937122345,
   -0.011404849588871002,
   0.06234355270862579,
   0.06365783512592316,
   -0.029701294377446175,
   -0.005723099689930677,
   0.005162716843187809,
   0.055933449417352676,
   0.006136275362223387,
   0.07008372247219086,
   0.0684361979

In [13]:
question = 'When was the house at 3524 Redwing Ct, Naperville, IL 60564 last sold and for what price?'
documents = db.similarity_search_with_relevance_scores(question, k=5, )
for i, doc in enumerate(documents):
    print(f'Document {i+1} ------------------------------------')
    print(doc[1])
    print(doc[0].page_content)
    print('\n\n')
    

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/embeddings